In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import os
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn import metrics
from sklearn.linear_model import Lasso, Ridge
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV

In [2]:
fifa = pd.read_csv("FIFA19data.csv", sep=r'\s*,\s*', engine='python')
fifa.head()

,ID,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,International Reputation,...,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes
0,158023,L. Messi,31,Argentina,94,94,FC Barcelona,�110.5M,�565K,5.0,...,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0
1,20801,Cristiano Ronaldo,33,Portugal,94,94,Juventus,�77M,�405K,5.0,...,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0
2,190871,Neymar Jr,26,Brazil,92,93,Paris Saint-Germain,�118.5M,�290K,5.0,...,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0
3,193080,De Gea,27,Spain,91,93,Manchester United,�72M,�260K,4.0,...,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0
4,192985,K. De Bruyne,27,Belgium,91,92,Manchester City,�102M,�355K,4.0,...,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0


In [3]:
fifa = fifa.drop('ID', 1)
fifa = fifa.drop('Name', 1)
fifa = fifa.drop('Nationality', 1)
fifa = fifa.drop('Club', 1)
fifa = fifa.drop('Value', 1)
fifa = fifa.drop('Wage', 1)
fifa = fifa.drop('Body Type', 1)
fifa = fifa.drop('Potential', 1)

In [4]:
for col in fifa.columns:
    fifa[col].fillna(value=fifa[col].mode()[0], inplace=True)

factors = ['International Reputation', 'Weak Foot', 'Skill Moves', 'Work Rate', 'Position', 'Contract Valid Until']

for var in factors:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(fifa[var], prefix=var)
    fifa = pd.concat([fifa,cat_list], axis = 1)
    fifa = fifa.drop(var, 1)

In [6]:
X = fifa.copy()
X = X.drop('Overall', 1)
Y = fifa.copy()
Y = Y['Overall']


0        94
1        94
2        92
3        91
4        91
         ..
18202    47
18203    47
18204    47
18205    47
18206    46
Name: Overall, Length: 18207, dtype: int64

In [8]:
X_train,X_test,y_train,y_test=train_test_split(X,Y, test_size=0.9, random_state=31)

In [12]:
Y

0        94
1        94
2        92
3        91
4        91
         ..
18202    47
18203    47
18204    47
18205    47
18206    46
Name: Overall, Length: 18207, dtype: int64

# Basic Linear Model

In [7]:
lm1 = LinearRegression()
lm1.fit(X_train, y_train)
lm1_predictions = lm1.predict(X_test)
lm1_r2 = r2_score(y_test,lm1_predictions)
print(lm1_r2)

0.8904970737556854


# CrossVal

In [8]:
#cv_predictions = cross_val_predict(lm1, fifa, Y, cv=5)
#cv_r2 = r2_score(Y,cv_predictions)
cv_predictions = cross_val_predict(lm1, X_test, y_test, cv=5)
cv_r2 = r2_score(y_test,cv_predictions)
print(cv_r2)

0.895506109826533


# Lasso

In [9]:
lasso = Lasso()
lasso.fit(X_train,y_train)
lasso1_predictions = lasso.predict(X_test)
train_score=lasso.score(X_train,y_train)
test_score=lasso.score(X_test,y_test)
coeff_used = np.sum(lasso.coef_!=0)
#r2_lasso1 = r2_score(y_test, lasso1_predictions)

In [10]:
print("training score:", train_score)
print("test score: ", test_score)
print("number of features used: ", coeff_used)
#print("test r2 score: ", r2_lasso1)

training score: 0.8584100702105248
test score:  0.8508221432029555
number of features used:  23


In [11]:
coef=lasso.sparse_coef_

In [12]:
print(coef)

  (0, 2)	0.00192659050046206
  (0, 3)	0.07914710326223977
  (0, 4)	0.06442788484646174
  (0, 6)	0.021205393979385357
  (0, 7)	0.00299853068605455
  (0, 10)	0.13767235145172543
  (0, 11)	0.01866613964871407
  (0, 12)	0.029243611479934685
  (0, 13)	0.003555402989815518
  (0, 14)	0.30281335982970176
  (0, 15)	-0.017870697860067432
  (0, 17)	0.002280524036873203
  (0, 19)	0.043706964823424656
  (0, 21)	0.0012858181881314334
  (0, 23)	-0.020360309756800456
  (0, 26)	0.11820667928613898
  (0, 27)	0.026291660322868674
  (0, 28)	0.0009654689660207915
  (0, 30)	0.07403002360850613
  (0, 31)	0.06307665967368914
  (0, 32)	0.007204357983149597
  (0, 33)	0.06872115360888857
  (0, 34)	0.06975255372994354


In [13]:
#If you are interested in checking adjusted R2
lm_train_score=lm1.score(X_train,y_train)
lm_test_score=lm1.score(X_test,y_test)
print("training score:", lm_train_score)
print("test score: ", lm_test_score)

lm_ra = 1-(1-lm_train_score)*((len(X_train)-1)/(len(X_train)-len(lm1.coef_)-1))
print(lm_ra)
print(lm_train_score)

lasso_ra = 1-(1-train_score)*((len(X_train)-1)/(len(X_train)-coeff_used-1))
print(lasso_ra)
print(train_score)

training score: 0.9035421515239555
test score:  0.8904970737556854
0.8966076450336329
0.9035421515239555
0.8565968361430649
0.8584100702105248


In [14]:
lasso = Lasso()

parameters = {'alpha': [1e-15, 1e-10, 1e-8, 1e-4, 1e-3,1e-2, 1, 5, 10, 20]}

lasso_regressor = GridSearchCV(lasso, parameters, cv = 5)

lasso_regressor.fit(X_train, y_train)

/Users/nandinibasu/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3372.952901155027, tolerance: 7.077029670329669
  positive)
/Users/nandinibasu/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3374.0470912585824, tolerance: 7.259854670329671
  positive)
/Users/nandinibasu/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3263.4799344132703, tolerance: 7.131133997252747
  positive)
/Users/nandinibasu/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did no

GridSearchCV(cv=5, error_score=nan,
             estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=1000, normalize=False, positive=False,
                             precompute=False, random_state=None,
                             selection='cyclic', tol=0.0001, warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': [1e-15, 1e-10, 1e-08, 0.0001, 0.001, 0.01, 1,
                                   5, 10, 20]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [15]:
lasso_regressor.best_params_

{'alpha': 0.01}

In [16]:
lasso_regressor.score(X_train,y_train)

0.900304099950968

In [17]:
lasso_regressor.score(X_test,y_test)

0.8898243448923757

In [18]:
lasso_regressor_predictions = lasso_regressor.predict(X_test)
lasso_regressor_r2 = r2_score(y_test,lasso_regressor_predictions)
lasso_regressor_r2

0.8898243448923757

In [19]:
coeff_used = np.sum(lasso_regressor.best_estimator_.coef_!=0)
print(coeff_used)

59


In [20]:
lasso2_predictions = lasso_regressor.predict(X_test)
r2_score(y_test,lasso2_predictions)

0.8898243448923757

# Ridge 

In [21]:
ridge_reg = Ridge(alpha=0)
ridge_reg.fit(X_train, y_train)
ridge_predictions = ridge_reg.predict(X_test)
train_score=ridge_reg.score(X_train,y_train)
test_score=ridge_reg.score(X_test,y_test)
print("training score:", train_score)
print("test score: ", test_score)

training score: 0.9032130400893218
test score:  -4.736594794869089e+21


In [22]:
# Calculation OOS R squared 
ridge_reg = Ridge()

parameters = {'alpha': [1e-15, 1e-10, 1e-8, 1e-4, 1e-3,1e-2, 1, 5, 10, 20]}

ridge_regressor = GridSearchCV(ridge_reg, parameters, cv = 5)

ridge_regressor.fit(X_train, y_train)

ridge_regressor.best_params_

ridge_regressor.score(X_train,y_train)

/Users/nandinibasu/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=9.59111e-18): result may not be accurate.
  overwrite_a=True).T
/Users/nandinibasu/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=8.17677e-18): result may not be accurate.
  overwrite_a=True).T
/Users/nandinibasu/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=7.98569e-18): result may not be accurate.
  overwrite_a=True).T
/Users/nandinibasu/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=8.29547e-18): result may not be accurate.
  overwrite_a=True).T
/Users/nandinibasu/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=7.92738e-18): result may not be ac

0.9027463664249874

# AIC BIC AICc

In [23]:
def AIC(y_true, y_hat, coeff_used):
    resid = y_true - y_hat
    sse = sum(resid**2)
    n = len(y_hat)
    return n*np.log(sse/n) + 2*coeff_used

def BIC(y_true, y_hat, coeff_used):
    resid = y_true - y_hat
    sse = sum(resid**2)
    n = len(y_hat)
    return n*np.log(sse/n) + np.log(n)*coeff_used

In [24]:
#aic and bic of simple linear model
aic_lm1 = AIC(y_test, lm1_predictions, (len(X_test.columns)+1))
print(aic_lm1)
bic_lm1 = BIC(y_test, lm1_predictions, (len(X_test.columns)+1))
print(bic_lm1)

27300.326786055328
28247.948750890515


In [25]:
#aic and bic of lasso model
aic_lasso2 = AIC(y_test, lasso2_predictions, (coeff_used+1))
print(aic_lasso2)
bic_lasso2 = BIC(y_test, lasso2_predictions, (coeff_used+1))
print(bic_lasso2)

27274.69198575777
27736.946602750544


In [26]:
def AICc(y_true, y_hat, coeff_used):
    resid = y_true - y_hat
    sse = sum(resid**2)
    n = len(y_hat)
    return n*np.log(sse/n) + 2*coeff_used*(len(y_true)/(len(y_true)-coeff_used-1))

In [27]:
aicc_lasso2=AICc(y_test, lasso2_predictions, (coeff_used+1))
print(aicc_lasso2)
aicc_lm1 = AICc(y_test, lm1_predictions, (len(X_test.columns)+1))
print(aicc_lm1)

27275.14035032962
27302.20245475114
